In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:
# Import data here
# col_names = []
data = []
data.head()

In [ ]:
# Feature selection

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
# Create the model
dtc = DecisionTreeClassifier() # optimization would happen here
dtc = dtc.fit(X_train, y_train)
y_predict = dtc.predict(X_test)

In [ ]:
# Accuracy
print("Accuracy of decision tree model is ", metrics.accuracy_score(y_test, y_predict))
print("F1 Score is ", metrics.f1_score(y_predict, y_test, average="weighted"))

In [ ]:
# Confusion matrix
# labels = ["cipher1", "cipher2", ...]
cm = metrics.confusion_matrix(y_test, y_predict)
display = metrics.ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
display.plot()